# EDA with SQL

### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 74.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=1b8f99faf4f09c184aa88923058136aed5df35f14867264bcb8671dfac9938f3
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


**Note:This below code is added to remove blank rows from table**


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [8]:
%sql select * from SPACEXTABLE limit 1;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)


In [9]:
%sql select distinct Launch_site from SPACEXTABLE ;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [10]:
%sql select launch_site from spacextable where launch_site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [17]:
%sql select sum(PAYLOAD_MASS__KG_) as total_payload from spacextable where customer='NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload
45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [18]:
%sql select avg(PAYLOAD_MASS__KG_) as average_payload from spacextable where Booster_Version='F9 v1.1';

 * sqlite:///my_data1.db
Done.


average_payload
2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [19]:
df.Landing_Outcome.unique()

array(['Failure (parachute)', 'No attempt', 'Uncontrolled (ocean)',
       'Controlled (ocean)', 'Failure (drone ship)',
       'Precluded (drone ship)', 'Success (ground pad)',
       'Success (drone ship)', 'Success', 'Failure', 'No attempt '],
      dtype=object)

In [20]:
%sql select min(date) from spacextable where Landing_Outcome='Success (ground pad)';

 * sqlite:///my_data1.db
Done.


min(date)
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [23]:
%sql select distinct customer from spacextable where Landing_Outcome='Success (drone ship)' and (PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000)

 * sqlite:///my_data1.db
Done.


Customer
SKY Perfect JSAT Group
SES
SES EchoStar


### Task 7




##### List the total number of successful and failure mission outcomes


In [26]:
df.Mission_Outcome.unique()

array(['Success', 'Failure (in flight)',
       'Success (payload status unclear)', 'Success '], dtype=object)

In [27]:
%sql select Mission_Outcome, count(*) as total_num from spacextable group by trim(Mission_Outcome);

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total_num
Failure (in flight),1
Success,99
Success (payload status unclear),1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [29]:
%sql select Booster_Version from spacextable where PAYLOAD_MASS__KG_=(select max(PAYLOAD_MASS__KG_) from spacextable)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [39]:
%sql select substr(Date, 6, 2) as month, Landing_Outcome, customer, launch_site from spacextable \
where Landing_Outcome='Failure (drone ship)' and substr(Date,0,5)='2015';

 * sqlite:///my_data1.db
Done.


month,Landing_Outcome,Customer,Launch_Site
10,Failure (drone ship),NASA (CRS),CCAFS LC-40
04,Failure (drone ship),NASA (CRS),CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [46]:
%sql select Landing_Outcome, count(*) as Counts from spacextable \
where date(Date) between date('2010-06-04') and date('2017-03-20') \
group by Landing_Outcome \
order by Counts

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Counts
Failure (parachute),1
Precluded (drone ship),1
Uncontrolled (ocean),2
Controlled (ocean),3
Failure (drone ship),5
Success (drone ship),5
Success (ground pad),5
No attempt,10
